In [198]:
# Project4 
# 1. 
!pip install folium
import folium
import requests
import pandas

arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

#drop data without location, which cannot be located on the map
arrest_table.dropna(subset=['lat','long'])
arrest_table

,arrest,age,race,sex,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,B,M,01/01/2011,00:01:00,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,B,M,01/01/2011,00:01:00,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,B,M,01/01/2011,00:04:00,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,B,M,01/01/2011,00:05:00,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,B,M,01/01/2011,00:05:00,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104522,13610388.0,32,B,M,12/31/2012,23:00:00,3300 Pulaski St,79-Other,3300 Pulaski Hw,1 5212,Handgun On Person || Handgun Violation,SOUTHEASTERN,223.0,Ellwood Park/Monument,"(39.2958396988, -76.5712467336)",39.295840,-76.571247
104523,13610389.0,27,B,M,12/31/2012,23:10:00,400 W Baltimore St,Unknown Offense,NaN,2 0055,Fail Obey Renble/Lawfl || Disorderly/Fto,CENTRAL,113.0,Downtown,"(39.2893323126, -76.6210021717)",39.289332,-76.621002
104524,13610377.0,34,B,M,12/31/2012,23:30:00,3800 Belair Rd,4E-Common Assault,3600 Belair Rd,1 1415,Asslt-Sec Degree || Assault,NORTHEASTERN,415.0,Belair-Edison,"(39.3252613570, -76.5689030474)",39.325261,-76.568903
104525,13610383.0,38,B,M,12/31/2012,23:40:00,2000 Mckean Ave,87-Narcotics,1700 Mckean Av,2A0696,Att-Cds Manuf/Dist-Narc || Distribution Cds,WESTERN,733.0,Mondawmin,"(39.3116837460, -76.6475011849)",39.311684,-76.647501


In [199]:
a = arrest_table.groupby(['race'])
print(a.describe())
b = arrest_table.groupby(['sex'])
print(b.describe())
#From the information below, we can know the types of races and sexes included in the data, and we'll create the map
#based on these types

       arrest                                                       \
        count          mean           std         min          25%   
race                                                                 
A       134.0  1.186072e+07  6.377593e+05  11127096.0  11260233.00   
B     52299.0  1.189901e+07  6.243486e+05  11126865.0  11266356.00   
H         1.0  1.259996e+07           NaN  12599962.0  12599962.00   
I       122.0  1.180369e+07  6.235856e+05  11130638.0  11230556.75   
U       871.0  1.179804e+07  6.106533e+05  11127198.0  11245550.50   
W      8516.0  1.197019e+07  1.763013e+06  11126920.0  11279413.25   

                                               age             ...        lat  \
             50%          75%         max    count       mean  ...        75%   
race                                                           ...              
A     11387057.0  12518843.25  12610343.0    149.0  32.013423  ...  39.313322   
B     12400965.0  12506236.00  13610460.0  53

In [200]:
#2. an interactive map of Baltimore
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
map_osm

In [203]:
import warnings
warnings.filterwarnings("ignore")
# 3.Due to the large amount of arrests, it will be messy to have popup for all data. As a result, I'll extract
# a random sample size of 500 to show the data. To do it, I'll firstly color markers corresponding to races,
# and then attach information in the topup

#---------------- direct representation of races by color -------------
from folium.plugins import MarkerCluster
#---------------------------------------------
# For types above, below are corresponding markers 
#A = Asian  -- red
#B = Black or African American -- pink
#H = Hispanic -- purple
#I = American Indian or Alaska Native -- green
#U = unknown -- blue
#W = White -- biege
#---------------------------------------------
#F = female = female_icon
#M = male = male_icon
#---------------------------------------------
marker_cluster = MarkerCluster().add_to(map_osm)
samples = arrest_table.sample(500)
for i in samples.index: 
        race_color = 'white'
        icon_type = 'info-sign'
        if arrest_table['sex'][i] == 'F': 
            icon_type = 'female'
        elif arrest_table['sex'][i] == 'M':
            icon_type = 'male'
        if arrest_table['race'][i] == 'A':
            race_color = 'red'
        elif arrest_table['race'][i] == 'B':
            race_color = 'pink'
        elif arrest_table['race'][i] == 'H':
            race_color = 'purple'
        elif arrest_table['race'][i] == 'I':
            race_color = 'green'
        elif arrest_table['race'][i] == 'U':
            race_color = 'blue'
        elif arrest_table['race'][i] == 'W':
            race_color = 'biege'
        folium.Marker(
            location=[arrest_table['lat'][i], arrest_table['long'][i]],
            popup="Arrest #: " + str(arrest_table['arrest'][i]) + ". \Arrest Location: " + arrest_table['arrestLocation'][i] + ".\ Age: " + str(arrest_table['age'][i]),
            icon = folium.Icon(color=race_color, icon=icon_type,prefix='fa')
        ).add_to(marker_cluster)

map_osm

In [202]:
#---------------- direct representation of sexes by color -------------
# For types above, below are corresponding markers 
#A = Asian  -- triangle
#B = Black or African American -- square
#H = Hispanic -- circle
#I = American Indian or Alaska Native -- diamond
#U = unknown -- star
#W = White -- spade
#---------------------------------------------
#F = female = red
#M = male = blue
#---------------------------------------------
map_osm2 = folium.Map(location=[39.29, -76.61], zoom_start=11)
marker_cluster2 = MarkerCluster().add_to(map_osm2)
samples = arrest_table.sample(500)
for i in samples.index: 
        sex_color = 'white'
        race_type = 'info-sign'
        if arrest_table['sex'][i] == 'F': 
            sex_color = 'red'
        elif arrest_table['sex'][i] == 'M':
            sex_color = 'blue'
        if arrest_table['race'][i] == 'A':
            race_type = 'triangle'
        elif arrest_table['race'][i] == 'B':
            race_type = 'square'
        elif arrest_table['race'][i] == 'H':
            race_type = 'circle'
        elif arrest_table['race'][i] == 'I':
            race_type = 'diamond'
        elif arrest_table['race'][i] == 'U':
            race_type = 'star'
        elif arrest_table['race'][i] == 'W':
            race_type = 'spade'
        folium.Marker(
            location=[arrest_table['lat'][i], arrest_table['long'][i]],
            popup="Arrest #: " + str(arrest_table['arrest'][i]) + ". \Arrest Location: " + arrest_table['arrestLocation'][i] + ".\ Age: " + str(arrest_table['age'][i]),
            icon = folium.Icon(color=sex_color, icon=race_type,prefix='fa')
        ).add_to(marker_cluster2)

map_osm2

In [204]:
# Then I want to create a heatmap by time showing the location and time (date) distribution of arrests, so 
# a sample of 10000 entries is created and other information like street address will be dropped.

import pandas as pd 
import folium.plugins as plugins
import datetime

# Create a map of "Stamen Toner" which makes heat map easy to see
map_osm2 = folium.Map(location=[39.29, -76.61],tiles='Stamen Toner', zoom_start=11)

# Create a sample with needed information
samples = arrest_table.sample(10000)
samples.reset_index()
dict = {'date':samples['arrestDate'], 
        'lat':samples['lat'], 
        'long':samples['long'],
        'weight': [1]*10000 } 
info = pd.DataFrame(dict) 
date_list = []
dates = []

# convert string times to datetime then sort (if not, 2012-01-01 will be before 2011-01-02)
times = pd.to_datetime(info.date, format='%m/%d/%Y').sort_values(ascending=True).unique().astype(str)

# get arrests data in each date according to the data sorted above, record the data and corresponding date
for date in times:
    date_list.append(info.loc[info.date == pd.to_datetime(date, format='%Y-%m-%d').strftime('%m/%d/%Y'),
                              ['lat', 'long','weight']].groupby(['lat', 'long']).sum().reset_index().values.tolist())
    dates.append(date[0:10])

# use HeatMapWithTime with the data and corresponding date as inputs to show arrests over time. 
hm = plugins.HeatMapWithTime(
    date_list,
    index=dates,
    auto_play=True,
    max_opacity=0.4
)

hm.add_to(map_osm2)
map_osm2

#Scroll the bar to see arrests at any time!

<h2>Writeup</h2>
<p>By observing the cluster points of markers, colors of points and shape of the sign as well as information in the popup from different random samples, we can see from the map that most arrest cases in Baltimore happened mostly around the center of the city. For all points of each sample, most criminals happened to be male instead of female, which can be clearly seen in the second map by color. In addition, most criminals happened to be black with differences in age or other attribue, which are represented pink in the first map.
From the third map, there is no clear relationship between dates and number of arrests but I think it's interesting to see :)
</p>